In [1]:
# 


In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
import src.config as config
from src.inference import (
    get_feature_store
)

from datetime import datetime, timezone, timedelta

def get_current_utc_hour_ceiled():
    """Get current UTC time ceiled to the next hour"""
    current = datetime.now(timezone.utc)
    if current.minute > 0 or current.second > 0:
        # Ceil to next hour
        current = current + timedelta(hours=1)

    # Reset minutes, seconds, and microseconds
    current = current.replace(minute=0, second=0, microsecond=0)
    return current

# Usage
current_hour = get_current_utc_hour_ceiled()
print(f"Current UTC hour (ceiled): {current_hour}")

Current UTC hour (ceiled): 2025-03-05 20:00:00+00:00


In [4]:
def fetch_predictions():
    current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour == current_hour)

    return query.read()

In [5]:
fs = get_feature_store()
fg = fs.get_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1
)

query = fg.select_all()

2025-03-05 14:02:23,696 INFO: Initializing external client
2025-03-05 14:02:23,701 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 14:02:24,846 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215683


In [6]:
df= query.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (53.09s) 


In [7]:
df

,pickup_location_id,predicted_demand,pickup_hour
0,176,0.0,2025-03-05 19:00:00+00:00
1,27,0.0,2025-03-05 19:00:00+00:00
2,152,3.0,2025-03-05 19:00:00+00:00
3,43,61.0,2025-03-05 19:00:00+00:00
4,262,16.0,2025-03-05 19:00:00+00:00
...,...,...,...
247,138,37.0,2025-03-05 19:00:00+00:00
248,200,0.0,2025-03-05 19:00:00+00:00
249,80,0.0,2025-03-05 19:00:00+00:00
250,78,0.0,2025-03-05 19:00:00+00:00


In [8]:
from datetime import datetime, timezone, timedelta

def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    # Extract components from the rounded-up time
    year = next_hour.year
    month = next_hour.month
    day = next_hour.day
    hour = next_hour.hour

    # Create date strings in YYYY-MM-DD format
    current_date = f"{year}-{month:02d}-{day:02d}"
    next_date = next_hour + timedelta(days=1)
    next_date_str = next_date.strftime("%Y-%m-%d")

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    # # First get the full day's data from Hopsworks
    # df = fg.filter(
    #     (fg.pickup_hour >= current_date) &
    #     (fg.pickup_hour < next_date_str)
    # ).read()

    # Then filter for next hour in the DataFrame
    hour_str = f"{current_date} {hour:02d}:00"
    df_hour = df[df['pickup_hour'] == hour_str]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Querying for date range: {current_date} to {next_date_str}")
    print(f"Filtering for hour: {hour_str}")
    print(f"Found {len(df_hour)} records")
    return df_hour

# Example usage:
predictions = fetch_next_hour_predictions()

2025-03-05 14:03:25,296 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 14:03:25,311 INFO: Initializing external client
2025-03-05 14:03:25,314 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 14:03:26,117 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215683
Current UTC time: 2025-03-05 19:03:25.293928+00:00
Next hour: 2025-03-05 20:00:00+00:00
Querying for date range: 2025-03-05 to 2025-03-06
Filtering for hour: 2025-03-05 20:00
Found 0 records


In [9]:
now = datetime.now(timezone.utc)

In [10]:
predictions

,pickup_location_id,predicted_demand,pickup_hour


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_location_id  252 non-null    int32                  
 1   predicted_demand    252 non-null    float64                
 2   pickup_hour         252 non-null    datetime64[us, Etc/UTC]
dtypes: datetime64[us, Etc/UTC](1), float64(1), int32(1)
memory usage: 5.1 KB


In [12]:
import pandas as pd
current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

In [13]:
dt = current_date.ceil('h')


In [ ]:
fs = get_feature_store()  
fg = fs.get_feature_group(  
    name=config.FEATURE_GROUP_MODEL_PREDICTION,  
    version=1  
)  
query = fg.select_all() 
# query = query.filter(fg.pickup_hour >= "2025-02-13 04:00")
query = query.filter(fg.pickup_hour > "2025-02-13 05:00:00")  
results = query.read()

2025-03-05 14:03:27,332 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 14:03:27,348 INFO: Initializing external client
2025-03-05 14:03:27,350 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 14:03:28,065 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215683


In [ ]:
fg.filter((fg.pickup_hour >= "2025-02-13") & (fg.pickup_hour < "2025-02-14")).read()

2025-03-05 13:56:55,657 ERROR: Flight returned timeout error, with message: Deadline Exceeded
Traceback (most recent call last):
  File "d:\Taxi_Project\nyc_taxi\taxi_venv\Lib\site-packages\hsfs\core\arrow_flight_client.py", line 394, in afs_error_handler_wrapper
    return func(instance, *args, **kw)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Taxi_Project\nyc_taxi\taxi_venv\Lib\site-packages\hsfs\core\arrow_flight_client.py", line 459, in read_query
    return self._get_dataset(
           ^^^^^^^^^^^^^^^^^^
  File "d:\Taxi_Project\nyc_taxi\taxi_venv\Lib\site-packages\retrying.py", line 56, in wrapped_f
    return Retrying(*dargs, **dkw).call(f, *args, **kw)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Taxi_Project\nyc_taxi\taxi_venv\Lib\site-packages\retrying.py", line 257, in call
    return attempt.get(self._wrap_exception)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Taxi_Project\nyc_taxi\taxi_venv\Lib\site-packages\retrying.py", line 301, i

FeatureStoreException: Could not read data using Hopsworks Query Service.

In [ ]:
results

,pickup_location_id,predicted_demand,pickup_hour
0,176,0.0,2025-03-05 19:00:00+00:00
1,27,0.0,2025-03-05 19:00:00+00:00
2,152,3.0,2025-03-05 19:00:00+00:00
3,43,61.0,2025-03-05 19:00:00+00:00
4,262,16.0,2025-03-05 19:00:00+00:00
...,...,...,...
247,138,37.0,2025-03-05 19:00:00+00:00
248,200,0.0,2025-03-05 19:00:00+00:00
249,80,0.0,2025-03-05 19:00:00+00:00
250,78,0.0,2025-03-05 19:00:00+00:00


In [ ]:
def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    fs = get_feature_store()
    fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)

    # Then filter for next hour in the DataFrame
    df_hour = df[df["pickup_hour"] == next_hour]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Found {len(df_hour)} records")
    return df_hour

In [ ]:
predictions = fetch_next_hour_predictions()

2025-03-05 13:57:11,459 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 13:57:11,467 INFO: Initializing external client
2025-03-05 13:57:11,468 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 13:57:12,175 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215683
Current UTC time: 2025-03-05 18:57:11.458646+00:00
Next hour: 2025-03-05 19:00:00+00:00
Found 252 records


In [ ]:
fs = get_feature_store()
fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)



2025-03-05 13:57:16,507 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 13:57:16,515 INFO: Initializing external client
2025-03-05 13:57:16,516 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 13:57:17,340 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215683


In [ ]:
df = fg.read()

Error: Reading data from Hopsworks, using Hopsworks Feature Query Service           


KeyboardInterrupt: 

In [ ]:
df["pickup_hour"].max()

Timestamp('2025-02-13 23:00:00+0000', tz='Etc/UTC')

In [ ]:
now = datetime.now(timezone.utc)
next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
print(next_hour)

2025-02-13 23:00:00+00:00


In [ ]:
df[df["pickup_hour"] == next_hour]

,pickup_location_id,predicted_demand,pickup_hour
34594,211,29.0,2025-02-13 23:00:00+00:00
34595,53,0.0,2025-02-13 23:00:00+00:00
34596,112,0.0,2025-02-13 23:00:00+00:00
34598,6,0.0,2025-02-13 23:00:00+00:00
34599,240,0.0,2025-02-13 23:00:00+00:00
...,...,...,...
34905,153,0.0,2025-02-13 23:00:00+00:00
34906,187,0.0,2025-02-13 23:00:00+00:00
34907,237,272.0,2025-02-13 23:00:00+00:00
34908,251,0.0,2025-02-13 23:00:00+00:00
